### Step 1: Install CANYA

Google Colab 환경에서 Conda 없이 바로 CANYA를 설치합니다. 필요한 종속성(`tensorflow`, `numpy`, `pandas`)은 Colab에 이미 설치되어 있으므로 별도 설치가 필요하지 않습니다.

In [1]:
# --- 1단계: Conda 설치 ---
# Colab에 Conda 환경을 관리할 Miniforge를 설치합니다.
!wget -q "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh"
!bash Miniforge3-Linux-x86_64.sh -bfp /usr/local
!conda config --set auto_update_conda false

# --- 2단계: CANYA 전용 가상 환경 생성 및 모든 의존성 설치 ---
# python 3.8을 기반으로 numpy, pandas, tensorflow 등 모든 요구사항을 한 번에 설치합니다.
!conda create -n canya_env python=3.8 numpy=1.19.5 pandas=1.3.3 tensorflow=2.6.0 keras=2.6.0 "protobuf<=3.20" matplotlib seaborn -c conda-forge -y

# --- 3단계: 가상 환경 내에 CANYA 설치 ---
# 위에서 만든 canya_env 환경 안에서 pip를 사용해 canya를 설치합니다.
!conda run -n canya_env pip install --no-cache-dir https://github.com/lehner-lab/canya/tarball/master

print("\n\n✅ 모든 설치가 성공적으로 완료되었습니다!")
print("CANYA는 'canya_env'라는 가상 환경에 설치되었습니다.")
print("아래 '가상 환경에서 CANYA 실행하기' 설명을 참고하여 다음 단계를 진행하세요.")

PREFIX=/usr/local
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2025.4.26-hbd8a1cb_0.conda
Extracting ld_impl_linux-64-2.43-h712a8e2_4.conda
Extracting libgomp-15.1.0-h767d61c_2.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.12-7_cp312.conda
Extracting tzdata-2025b-h78e105d_0.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-15.1.0-h767d61c_2.conda
Extracting c-ares-1.34.5-hb9d3cd8_0.conda
Extracting libexpat-2.7.0-h5888daf_0.conda
Extracting libffi-3.4.6-h2dba641_1.conda
Extracting libgcc-ng-15.1.0-h69a702a_2.conda
Extracting libiconv-1.18-h4ce23a2_1.conda
Extracting liblzma-5.8.1-hb9d3cd8_1.conda
Extracting libstdcxx-15.1.0-h8f9b012_2.conda
Extracting libzlib-1.3.1-hb9d3cd8_2.conda
Extracting ncurses-6.5-h2d0b736_3.conda
Extracting openssl-3.5.0-h7b32b05_1.conda
Extracting reproc-14.2.5.post0-hb9d3cd8_0.conda
Extracting bzip2-1.0.8-h4bc722e_7.conda
Extracting cpp-expected-1.1.0-hff21bea_1

### Step 2: Prepare Input File

`canya_input.txt`라는 입력 파일을 생성합니다. 형식은 CANYA에서 요구하는 형식에 맞게 구성해야 합니다 (예: 탭 구분 서열 리스트).

In [2]:
# 헤더("ID\tSequence\n")를 제거하고 파일을 다시 생성합니다.
with open("canya_input.txt", "w") as f:
    f.write("Seq1\tCASSIRSSYEQYF\n")
    f.write("Seq2\tCASSLGQGAETLYF\n")

print("헤더가 제거된 'canya_input.txt' 파일을 생성했습니다.")

헤더가 제거된 'canya_input.txt' 파일을 생성했습니다.


### Step 3: Run CANYA (Basic Mode)

In [3]:
# !conda run -n canya_env 접두사를 붙여서 실행합니다.
print("--- Running Basic Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output basic_out.txt

print("\n--- Running Summarize Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output summarized_out.txt --summarize mean

print("\n--- Running Ensemble Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output ensemble_out.txt --summarize mean --mode ensemble

print("\n\n✅ 모든 실행이 완료되었습니다. 출력 파일을 확인하세요.")

--- Running Basic Mode ---
Loaded model models/model_weights1.h5
Read input
Finished 100% (2/2) of sequences

2025-07-12 11:28:03.756643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-12 11:28:03.892335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-12 11:28:03.892674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-07-12 11:28:03.893274: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical op

### Step 4: Visualize Results

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

try:
    # 앙상블 모드 결과 파일을 읽어옵니다.
    df = pd.read_csv('ensemble_out.txt', sep='\t')
    df_sorted = df.sort_values('Mean_Score', ascending=False).reset_index(drop=True)

    # 그래프를 생성합니다.
    plt.figure(figsize=(10, 6))
    sns.barplot(x='ID', y='Mean_Score', data=df_sorted, palette='viridis', edgecolor='black')

    # 에러바(표준편차)를 추가합니다.
    plt.errorbar(x=df_sorted.index, y=df_sorted['Mean_Score'], yerr=df_sorted['Std_Dev_Score'], fmt='none', c='black', capsize=5)

    plt.title('CANYA Nucleation Score (Ensemble Mode)', fontsize=16)
    plt.xlabel('Sequence ID')
    plt.ylabel('Mean ± Std Dev')
    plt.ylim(0, 1)
    plt.tight_layout()

    # 생성된 그래프를 이미지 파일로 저장합니다.
    plt.savefig('canya_results.png')
    print('Result plot saved to canya_results.png')

except FileNotFoundError:
    print('Error: ensemble_out.txt not found. Please ensure the previous CANYA command ran successfully.', file=sys.stderr)
except Exception as e:
    print(f'An error occurred: {e}', file=sys.stderr)

Overwriting visualize.py


In [6]:
!pwd

/content
